<a href="https://colab.research.google.com/github/juanFFlorezM/Extraccion_de_datos/blob/main/Analisis_Anova_PS_12_instructores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
# Paso 1: Cargar el archivo Excel
archivo_excel = 'Qt_3PS_excel.xlsx'  # Cambia esto por la ruta a tu archivo
datos = pd.read_excel(archivo_excel)

In [3]:
# Paso 2: Verifica la estructura de los datos
print("Datos cargados:")
print(datos.head())

Datos cargados:
  Instructor  S1  S2  S3
0         Q1   7   7   7
1         Q1   5   5   5
2         Q1   1   1   1
3         Q1   5   6   5
4         Q1   6   6   6


In [4]:
# Paso 3: Transformar los datos a formato largo
# Usamos pd.melt para transformar el DataFrame
datos_largos = pd.melt(datos, id_vars=['Instructor'], value_vars=['S1', 'S2', 'S3'],
                       var_name='Pregunta', value_name='Valor')

In [5]:
# Paso 4: Verifica la transformación
print("\nDatos transformados a formato largo:")
print(datos_largos.head())


Datos transformados a formato largo:
  Instructor Pregunta  Valor
0         Q1       S1      7
1         Q1       S1      5
2         Q1       S1      1
3         Q1       S1      5
4         Q1       S1      6


In [6]:
# Paso 5: Realizar ANOVA
modelo = ols('Valor ~ C(Instructor)', data=datos_largos).fit()  # Ajusta el modelo OLS
anova_resultado = anova_lm(modelo, typ=2)                      # Realiza ANOVA
print("\nResultados de ANOVA:")
print(anova_resultado)


Resultados de ANOVA:
                   sum_sq     df         F    PR(>F)
C(Instructor)   62.864785   11.0  3.643621  0.000052
Residual       912.860805  582.0       NaN       NaN


In [7]:
# Paso 6: Interpretar el resultado ANOVA
p_value_anova = anova_resultado['PR(>F)'][0]
if p_value_anova < 0.05:
    print("Hay diferencias significativas entre los instructores (p < 0.05).")

Hay diferencias significativas entre los instructores (p < 0.05).


<ipython-input-7-532034d692fc>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value_anova = anova_resultado['PR(>F)'][0]


In [8]:
# Paso 7: Realizar la prueba post-hoc Tukey HSD si hay diferencias significativas
if p_value_anova < 0.05:
    tukey_resultado = pairwise_tukeyhsd(endog=datos_largos['Valor'], groups=datos_largos['Instructor'], alpha=0.05)
    print("\nResultados de Tukey HSD:")
    print(tukey_resultado)
else:
    print("No hay diferencias significativas entre los instructores (p >= 0.05).")


Resultados de Tukey HSD:
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
    Q1    Q10   0.1343    1.0 -0.6056  0.8741  False
    Q1    Q11  -0.4663  0.748 -1.2642  0.3317  False
    Q1    Q12   0.1912 0.9998 -0.6258  1.0083  False
    Q1     Q2   0.1038    1.0 -0.6484  0.8559  False
    Q1     Q3   0.5861 0.5842 -0.3069  1.4792  False
    Q1     Q4   0.4459  0.525 -0.2089  1.1006  False
    Q1     Q5   0.8323 0.0415  0.0152  1.6494   True
    Q1     Q6  -0.2866 0.9951 -1.1505  0.5773  False
    Q1     Q7  -0.1917 0.9997 -0.9726  0.5893  False
    Q1     Q8  -0.0652    1.0 -0.8823  0.7519  False
    Q1     Q9   0.2242 0.9968 -0.4848  0.9332  False
   Q10    Q11  -0.6005 0.4559  -1.446   0.245  False
   Q10    Q12    0.057    1.0 -0.8066  0.9206  False
   Q10     Q2  -0.0305    1.0  -0.833   0.772  False
   Q10     Q3   0.4519 0.9143  -0.484  1.3877  False
   Q10     Q4   0.31

In [9]:
# Este es un codigo que me permite realizar un diagrama de cajas y bigotes con la data en datos_largos
import plotly.express as px
fig = px.box(datos_largos, x='Instructor', y='Valor', title='Diagrama de Caja y Bigotes')
fig.show()

In [7]:
# El siguiente codigo genera una tabla comparativa de los valores de mediana y media de la presencia social de los 12 instructores presente en datos_largos
tabla_comparativa = datos_largos.groupby('Instructor')['Valor'].agg(['median', 'mean'])
print(tabla_comparativa)

            median      mean
Instructor                  
Q1             6.0  5.680556
Q10            6.0  5.814815
Q11            5.0  5.214286
Q12            6.0  5.871795
Q2             6.0  5.784314
Q3             6.0  6.266667
Q4             7.0  6.126437
Q5             7.0  6.512821
Q6             5.0  5.393939
Q7             6.0  5.488889
Q8             6.0  5.615385
Q9             6.0  5.904762


In [10]:
#Este codigo organiza de los datos de media y mediana de  la presencia social de los instructores presente en la estructura datos_largos de mayor a menor valor
tabla_comparativa = tabla_comparativa.sort_values(by='mean', ascending=False)
print(tabla_comparativa)

            median      mean
Instructor                  
Q5             7.0  6.512821
Q3             6.0  6.266667
Q4             7.0  6.126437
Q9             6.0  5.904762
Q12            6.0  5.871795
Q10            6.0  5.814815
Q2             6.0  5.784314
Q1             6.0  5.680556
Q8             6.0  5.615385
Q7             6.0  5.488889
Q6             5.0  5.393939
Q11            5.0  5.214286


Esta seccion del archivo hace un analisis de atipicos y vuelve a dibujar el diagrama de cajas y bigotes

Metodo1_ Desviacion Estandar

In [10]:
# Este código en Python calcula los limites superior e inferior de la columna valor de datos_largos usando el metodo
# de la desviacion estandar e imprime los valores
media = datos_largos['Valor'].mean()
desviacion_estandar = datos_largos['Valor'].std()
limite_superior = media + 3 * desviacion_estandar
limite_inferior = media - 3 * desviacion_estandar
print("Limite Superior:", limite_superior)
print("Limite Inferior:", limite_inferior)

Limite Superior: 9.668066385103426
Limite Inferior: 1.971664254627215


In [14]:
#Generamos un dataframe datos_largos_f con el resultado de aplicar el metodo de desviacion estandar a datos largos
datos_largos_da = datos_largos[(datos_largos['Valor'] >= limite_inferior) & (datos_largos['Valor'] <= limite_superior)]

In [15]:
# corroboramos los nulos de datos_largos
datos_largos.isnull().sum()

,0
Instructor,0
Pregunta,0
Valor,0


In [16]:
#Corroboramos los nulos de datos_largos_f
datos_largos_da.isnull().sum()

,0
Instructor,0
Pregunta,0
Valor,0


Metodo2_ IQR

In [17]:
#Se calculan los limites superior e inferior de datos_largos usando el metodo de cuartiles y se imprime el valor de los limites
Q1 = datos_largos['Valor'].quantile(0.25)
Q3 = datos_largos['Valor'].quantile(0.75)
IQR = Q3 - Q1
limite_superior_iqr = Q3 + 1.5 * IQR
limite_inferior_iqr = Q1 - 1.5 * IQR
print("Limite Superior:", limite_superior_iqr)
print("Limite Inferior:", limite_inferior_iqr)

Limite Superior: 10.0
Limite Inferior: 2.0


In [18]:
#se genera un dataframa datos_largos_iqr donde se guarda el resultado aplicar el metodo de cuartiles a datos_largos
datos_largos_iqr = datos_largos[(datos_largos['Valor'] >= limite_inferior_iqr) & (datos_largos['Valor'] <= limite_superior_iqr)]

In [19]:
# se listan los valores nulos presentes en datos_largos_iqr
datos_largos_iqr.isnull().sum()

,0
Instructor,0
Pregunta,0
Valor,0


Conclusion: Despues de aplicar limpieza de atipicos con DE o IQR el resultado es el mismo al dataframe original.

Ahora se dibujan en una misma grafica las cajas y bigotes de los tres dataframes:

In [20]:
# Este codigo genera en una misma figura los diagramas de cajas y bigotes de datos_largos, datos_largos_da y datos_largos_iqr
fig = px.box(datos_largos, x='Instructor', y='Valor', title='Boxplot original')
fig.add_trace(px.box(datos_largos_da, x='Instructor', y='Valor', title='Boxplot DE').data[0])
fig.add_trace(px.box(datos_largos_iqr, x='Instructor', y='Valor', title='Boxplot IQR').data[0])
fig.show()


Conclusion: aunque para la mayoria de los instrcutores no hay cambio, para algunos el filtraje de atipicos si dio resultados diferentes: Q1,Q4,Q9 y Q11.

Entonces: se decide realizar un ANOVA a cada estrcutura de datos con atipicos y sin atipicos por los dos metodos DE e IQR

In [21]:
# El siguiente codigo realiza un analisis anova a los tres dataframes: datos_largos, datos_largos_da y datos_largos_iqr e imprime para cada uno de
# ellos si existen o no existen diferencias significativas entre los instructores.
modelo1 = ols('Valor ~ C(Instructor)', data=datos_largos).fit()
anova_resultado1 = anova_lm(modelo1, typ=2)
print("\nResultados de ANOVA original:")
print(anova_resultado1)
modelo2 = ols('Valor ~ C(Instructor)', data=datos_largos_da).fit()
anova_resultado2 = anova_lm(modelo2, typ=2)
print("\nResultados de ANOVA DE:")
print(anova_resultado2)
modelo3 = ols('Valor ~ C(Instructor)', data=datos_largos_iqr).fit()
anova_resultado3 = anova_lm(modelo3, typ=2)
print("\nResultados de ANOVA IQR:")
print(anova_resultado3)
#


Resultados de ANOVA original:
                   sum_sq     df         F    PR(>F)
C(Instructor)   62.864785   11.0  3.643621  0.000052
Residual       912.860805  582.0       NaN       NaN

Resultados de ANOVA DE:
                   sum_sq     df         F        PR(>F)
C(Instructor)   70.784919   11.0  5.504833  2.170593e-08
Residual       668.651725  572.0       NaN           NaN

Resultados de ANOVA IQR:
                   sum_sq     df         F        PR(>F)
C(Instructor)   70.784919   11.0  5.504833  2.170593e-08
Residual       668.651725  572.0       NaN           NaN


De acuerdo al test de anova hay diferencias significativas en las tres estrcutura de datos, asi que realizamos un test de tukey para ver si las parejas en cada estrutura son las mismas. Otra cosa: el filtrado de ambos metodos elimino 10 datos pero no los coloco como nulos.

In [25]:
# El siguiente codigo realiza el test de tukey a datos_largos_iqr e imprime el resultado
tukey_resultado = pairwise_tukeyhsd(endog=datos_largos['Valor'], groups=datos_largos['Instructor'], alpha=0.05)
print("\nResultados de Tukey HSD: datos_largos")
print(tukey_resultado)


Resultados de Tukey HSD: datos_largos
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
    Q1    Q10   0.1343    1.0 -0.6056  0.8741  False
    Q1    Q11  -0.4663  0.748 -1.2642  0.3317  False
    Q1    Q12   0.1912 0.9998 -0.6258  1.0083  False
    Q1     Q2   0.1038    1.0 -0.6484  0.8559  False
    Q1     Q3   0.5861 0.5842 -0.3069  1.4792  False
    Q1     Q4   0.4459  0.525 -0.2089  1.1006  False
    Q1     Q5   0.8323 0.0415  0.0152  1.6494   True
    Q1     Q6  -0.2866 0.9951 -1.1505  0.5773  False
    Q1     Q7  -0.1917 0.9997 -0.9726  0.5893  False
    Q1     Q8  -0.0652    1.0 -0.8823  0.7519  False
    Q1     Q9   0.2242 0.9968 -0.4848  0.9332  False
   Q10    Q11  -0.6005 0.4559  -1.446   0.245  False
   Q10    Q12    0.057    1.0 -0.8066  0.9206  False
   Q10     Q2  -0.0305    1.0  -0.833   0.772  False
   Q10     Q3   0.4519 0.9143  -0.484  1.3877  False
   Q10 

In [27]:
#El siguiente codigo dibuja una figura con los diagramas de cajas y bigotes de los intructores de datos_largos_iqr, de datos_largos_da y de datos_largos
fig = px.box(datos_largos_iqr, x='Instructor', y='Valor', title='Boxplot IQR')
fig.show()
fig = px.box(datos_largos_da, x='Instructor', y='Valor', title='Boxplot DE')
fig.show()
fig = px.box(datos_largos, x='Instructor', y='Valor', title='Boxplot original')
fig.show()

Evidentemente hay diferencias en los test de Tukey de datosoriginal con respecto a los datos filtrados con DE o IQR. Pero debo averiguar si la perdida de esos datos outliers, al no poder reemplazarlos por la
media o mediana esta afectando el resultado del test de tukey

In [28]:
datos_largos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594 entries, 0 to 593
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Instructor  594 non-null    object
 1   Pregunta    594 non-null    object
 2   Valor       594 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 14.1+ KB


In [29]:
datos_largos_da.info()

<class 'pandas.core.frame.DataFrame'>
Index: 584 entries, 0 to 593
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Instructor  584 non-null    object
 1   Pregunta    584 non-null    object
 2   Valor       584 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 18.2+ KB


In [30]:
datos_largos_iqr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 584 entries, 0 to 593
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Instructor  584 non-null    object
 1   Pregunta    584 non-null    object
 2   Valor       584 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 18.2+ KB


In [31]:
#este codigo imprime o despliega los valores nulos de datos_largos_da
datos_largos_da.isnull().sum()

,0
Instructor,0
Pregunta,0
Valor,0


In [33]:
#este codigo despliega los valores de la columna Instructor con valor "Q1" presente en el dataframes datos_largos
datos_largos_da[datos_largos_da['Instructor'] == 'Q1']

,Instructor,Pregunta,Valor
0,Q1,S1,7
1,Q1,S1,5
3,Q1,S1,5
4,Q1,S1,6
5,Q1,S1,6
...,...,...,...
415,Q1,S3,7
416,Q1,S3,6
417,Q1,S3,6
418,Q1,S3,6


In [47]:
#Este codigo despliega los indices de los valores de la columna Instructor "Q1" presentes en datos_largos que no se encuentran en datos_largos_da
datos_largos[~datos_largos.index.isin(datos_largos_da.index)][datos_largos['Instructor'] == 'Q12'].index

<ipython-input-47-95c93d16ef59>:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



Index([], dtype='int64')

Cuando en el codigo anterior se reemplza q1 hasta q12: estos son los indices que cambian q1[2,200,398], Q4[65, 263, 461], Q9[346,349,544] y Q11[380]

In [51]:
#Ahora debo calcular el valor de mediana de la columna Instructor "Q1" presente en datos_largos
datos_largos[datos_largos['Instructor'] == 'Q11']['Valor'].median()

5.0

Con el codigo anterior pude sacar los valores de mediana de Q1[6.0], Q4[7.0], Q9[6.0] y Q11[5.0]

In [56]:
#Ahora necesito copiar en un dataframe dl_q1(4,9 y 11) copiar los registros con los indices 2, 200 y 398 de datos_largos e imprimir dl_q1
dl_q11 = datos_largos.loc[[380]]
print(dl_q11)

    Instructor Pregunta  Valor
380        Q11       S2      1


In [60]:
#Ahora debo cambiar todos los valores presentes en la columna Valor de dl_q1(4,9 y 11) por el valor 6.0(7,6 y 5)
dl_q11['Valor'] = 5.0
print(dl_q11)

    Instructor Pregunta  Valor
380        Q11       S2    5.0


In [61]:
# ahora necesito concatenar dl_q1, dl_q4, dl_q9, dl_q11 y datos_largos_da en un nuevo dataframe datos_largo_dac.
datos_largo_dac = pd.concat([datos_largos_da, dl_q1, dl_q4, dl_q9, dl_q11])
print(datos_largo_dac)

    Instructor Pregunta  Valor
0           Q1       S1    7.0
1           Q1       S1    5.0
3           Q1       S1    5.0
4           Q1       S1    6.0
5           Q1       S1    6.0
..         ...      ...    ...
461         Q4       S3    7.0
346         Q9       S2    6.0
349         Q9       S2    6.0
544         Q9       S3    6.0
380        Q11       S2    5.0

[594 rows x 3 columns]


In [62]:
#Se compara la info de datos_largos con datos_largos_dac
datos_largos.info()
datos_largo_dac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594 entries, 0 to 593
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Instructor  594 non-null    object
 1   Pregunta    594 non-null    object
 2   Valor       594 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 14.1+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 594 entries, 0 to 380
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Instructor  594 non-null    object 
 1   Pregunta    594 non-null    object 
 2   Valor       594 non-null    float64
dtypes: float64(1), object(2)
memory usage: 18.6+ KB


In [63]:
# el siguiente codigo compara si el dataframe datos_largos es igual o no lo es al dataframe datos_largos_dac
datos_largos.equals(datos_largo_dac)

False

Ahora que ya tenemos un nuevo dataframe datos_largos_dac en el cual los atipicos de Q1, Q4, Q9 y Q11 se han reemplazado por la mediana. Se procede a comprobar si hay cambios en las cajas y bigotes y sobre todo en el Anova y Test de Tukey

In [64]:
# Este codigo permite comparar en una misma figura los diagramas de cajas y bigotes de los dataframes datos_largos y datos_largos_dac
fig = px.box(datos_largos, x='Instructor', y='Valor', title='Boxplot original')
fig.add_trace(px.box(datos_largo_dac, x='Instructor', y='Valor', title='Boxplot DEC').data[0])
fig.show()

Efectivamente hay diferencias, ahora veremos que nos dice el anova y posterior, espero, test de tukey de datos_largos_dac

In [65]:
#Este codigo permite realizar una prueba anova de diferencias significativas de la columna Instructor en datos_largos_dac
# e imprime los resultados
modelo4 = ols('Valor ~ C(Instructor)', data=datos_largo_dac).fit()
anova_resultado4 = anova_lm(modelo4, typ=2)
print("\nResultados de ANOVA DEC:")
print(anova_resultado4)


Resultados de ANOVA DEC:
                   sum_sq     df         F        PR(>F)
C(Instructor)   73.673655   11.0  5.815891  5.638466e-09
Residual       670.233752  582.0       NaN           NaN


Efectivamente hay diferencias significativas, ahora averiguaremos entre que parejas de instructores para datos_largos_dac

In [67]:
#este codigo permite generar un test de tukey a datos_largos_dac
tukey_resultado = pairwise_tukeyhsd(endog=datos_largo_dac['Valor'], groups=datos_largo_dac['Instructor'], alpha=0.05)
print("\nResultados de Tukey HSD: datos_largos_dac")
print(tukey_resultado)


Resultados de Tukey HSD: datos_largos_dac
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
    Q1    Q10  -0.0741    1.0  -0.708  0.5599  False
    Q1    Q11  -0.5794 0.1911 -1.2631  0.1044  False
    Q1    Q12  -0.0171    1.0 -0.7172   0.683  False
    Q1     Q2  -0.1046    1.0 -0.7491  0.5399  False
    Q1     Q3   0.3778 0.9014 -0.3874   1.143  False
    Q1     Q4   0.4444 0.2823 -0.1166  1.0055  False
    Q1     Q5   0.6239 0.1351 -0.0762  1.3241  False
    Q1     Q6  -0.4949 0.5547 -1.2352  0.2453  False
    Q1     Q7     -0.4 0.7193 -1.0692  0.2692  False
    Q1     Q8  -0.2735 0.9812 -0.9736  0.4266  False
    Q1     Q9    0.254 0.9684 -0.3535  0.8615  False
   Q10    Q11  -0.5053 0.4859 -1.2298  0.2192  False
   Q10    Q12    0.057    1.0  -0.683   0.797  False
   Q10     Q2  -0.0305    1.0 -0.7181  0.6571  False
   Q10     Q3   0.4519 0.7898   -0.35  1.2537  False
   

Conclusion: el analisis de tukey de datos_largos_dac donde los atipicos eliminados con DE y reemplazados por la mediana, dieron el mismo conjunto parejas que el analisis de tukey de datos_largos_da. Fueron en total 10 parejas, 4 mas que las obtenidas con datos_largos(donde no se han retirado los atipicos).Estas son las 10 comparaciones:

Q11-(Q3,Q4,Q5,Q9)
Q4-(Q6,Q7,Q8)
Q5-(Q6,Q7,Q8)